# 📘 Sensor Calibration: Polynomial and Linear Fits> Calibrate sensors using polynomial regression⏱️ **15-20 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Engineering**---

## 🔬 Domain Background**Model:** $Output = a_0 + a_1 \cdot Input + a_2 \cdot Input^2 + ...$**Applications:** Temperature sensors, pressure transducers, load cells---

## Setup

In [ ]:
import numpy as npimport jax.numpy as jnpfrom nlsq import curve_fitimport matplotlib.pyplot as pltdef quadratic(x, a, b, c):    return a + b*x + c*x**2

## Generate Calibration Data

In [ ]:
# True sensor response (nonlinear)a_true, b_true, c_true = 0.5, 2.0, -0.01x_input = np.linspace(0, 100, 20)  # Input signaly_true = quadratic(x_input, a_true, b_true, c_true)y_output = y_true + np.random.normal(0, 0.5, len(x_input))plt.plot(x_input, y_output, 'o', label='Calibration data')plt.xlabel('Input Signal')plt.ylabel('Sensor Output')plt.legend()plt.show()

## Fit Calibration Curve

In [ ]:
popt, pcov = curve_fit(quadratic, x_input, y_output, p0=[0, 2, 0])a_fit, b_fit, c_fit = poptperr = np.sqrt(np.diag(pcov))print(f'Calibration equation:')print(f'  Output = {a_fit:.3f} + {b_fit:.3f}*Input + {c_fit:.5f}*Input²')print(f'\nParameter uncertainties:')print(f'  σ(a) = {perr[0]:.3f}')print(f'  σ(b) = {perr[1]:.3f}')print(f'  σ(c) = {perr[2]:.5f}')

## Validate Calibration

In [ ]:
# Test with new datax_test = np.array([25, 50, 75])y_predicted = quadratic(x_test, *popt)print('\nCalibration validation:')for x, y in zip(x_test, y_predicted):    print(f'  Input {x} → Output {y:.2f}')

## Key Insights1. **Quadratic captures sensor nonlinearity**2. **Residuals indicate calibration quality**3. **Regular recalibration** needed for drift---